In [ ]:
import numpy as np
import xarray as xr
import sys
sys.path.append('../')
from xdispersion import RelativeDispersion
import matplotlib.cm as cm
from matplotlib import cm
import matplotlib.pyplot as plt
import pandas as pd

save=False  #save figures or not
namedata = "ExpN"
drifters = xr.open_dataset('./ExpN/fort.1001.np4000nt6000.nc')
#drifters = xr.open_dataset('./QGdaphne-E12e-8/fort.2001.np4000nt3000.nc')


rd = RelativeDispersion(drifters, ragged=False, ID='tracer',
                        xpos='x', uvel='ux', time='time',
                        ypos='y', vvel='uy', coord='polar')

pairs = rd.get_all_pairs()
origin_pairs = rd.get_original_pairs(pairs, r0=[0.005, 0.01])

rx, ry, rxy, r, rpb = rd.separation_measures(origin_pairs) # separations of different components
#du, dv, dul, dut, vmi, vmj, uv = rd.velocity_measures(origin_pairs) # velocities of different components


In [ ]:
from xdispersion import gen_rbins, rel_disp, rel_diff, kurtosis, cumul_inv_sep_time, famp_growth_rate, fsize_lyap_exp

r2   = rel_disp(r, order=2, mean_at='const-t') #order=2 so 2nd moment of separation
r2x   = rel_disp(rx, order=2, mean_at='const-t') #radial
r2y   = rel_disp(ry, order=2, mean_at='const-t') #zonal (arclength)

r2yb = r2-r2x #estimate of zonal


In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(10, 3))  # (rows, cols)
ax = axes[0]  # pick the first subplot (dispersion)
#r2.plot.line(xscale='log', yscale='log', figsize=(6,3.5), marker='o', color='b', markersize=2, lw=0.5)
ax.scatter(r2.rtime,r2.values,marker='x',s=5,color='b',label="full dispersion")
ax.scatter(r2x.rtime,r2x.values,marker='x',s=5,color='y',label="radial dispersion")
ax.scatter(r2y.rtime,r2y.values,marker='x',s=5,color='r',label="radial dispersion")
ax.scatter(r2yb.rtime,r2yb.values,marker='x',s=5,color='r',label="radial dispersion")
ax.set_ylim(1e-5,max(r2.values)*100)
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel(r"$t$ (s)")
ax.set_ylabel(r"$\langle r^2 \rangle$ (m$^2$)")
ax.set_title("Relative Dispersion")

axes[1].scatter(np.sqrt(r2),r2y/r2x,s=5,color='b')
axes[1].scatter(np.sqrt(r2),r2yb/r2x,s=5,color='y')
axes[1].set_ylabel(r"$\langle r_z^2 \rangle / \langle r_r^2 \rangle$")
axes[1].set_xlabel(r"$r_{\rm rms}$ (m)")
axes[1].set_xscale('log')
#axes[1].set_yscale('log')

axes[2].scatter(r2.rtime,r2y/r2x,s=5,color='b')
axes[2].scatter(r2.rtime,r2yb/r2x,s=5,color='y')
axes[2].set_ylabel(r"$\langle r_z^2 \rangle / \langle r_r^2 \rangle$")
axes[2].set_xlabel(r"$t$ (s)")
axes[2].set_xscale('log')
#axes[2].set_yscale('log')

plt.tight_layout(rect=[0, 0, 1, 0.93])  # leave space at top for labels
plt.show()

